# Naukri.com Webscraping for Demand/Supply Analysis


In [87]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from selenium import webdriver
import time
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [83]:
def scrape_naukri(city,experience,designation):
    Applicants=[]
    Openings=[]
    Comp_name=[]
    #sum_applicants=0
    #sum_openings=0
    driver = webdriver.Chrome()
    driver.get("https://www.naukri.com/")
    driver.implicitly_wait(10)
    original_window = driver.current_window_handle
    
    designation_input=driver.find_element(By.CLASS_NAME, value='suggestor-input')
    designation_input.send_keys(designation)
    
    city_input=driver.find_elements(By.CLASS_NAME, value='suggestor-input')[1]
    city_input.send_keys(city)
    
    experience_input=driver.find_element(By.ID, value='expereinceDD').click()
    experience_list=driver.find_element(By.CLASS_NAME , value='dropdown').find_elements(By.TAG_NAME,'li')
    
    for i in experience_list:
        if experience in i.get_attribute("innerText"):
            i.click()
            break
    
    driver.find_element(By.CLASS_NAME,'qsbSubmit').click()
    filters=driver.find_elements(By.CLASS_NAME,'styles_filterHeading___hZQx')
    filter_options=driver.find_elements(By.CLASS_NAME,'styles_filterOptns__1vq77')
    for i in range(len(filters)):
        if filters[i].get_attribute("innerText")=="Department":
            options=filter_options[i].find_elements(By.CLASS_NAME,'styles_txtLbl__Pl_48')
            for j in range(len(options)):
                if options[j].find_element(By.CLASS_NAME,'styles_filterLabel__jRP04').get_attribute("innerText")=="Engineering - Software & QA":
                    filter_options[i].find_elements(By.CLASS_NAME,'ni-icon-unchecked')[j].click()
                    break
    time.sleep(5)       
    filters=driver.find_elements(By.CLASS_NAME,'styles_filterHeading___hZQx')
    filter_options=driver.find_elements(By.CLASS_NAME,'styles_filterOptns__1vq77')
    for i in range(len(filters)):
        if filters[i].get_attribute("innerText")=="Industry":
            options=filter_options[i].find_elements(By.CLASS_NAME,'styles_txtLbl__Pl_48')
            for j in range(len(options)):
                if options[j].find_element(By.CLASS_NAME,'styles_filterLabel__jRP04').get_attribute("innerText")=="Software Product":
                    filter_options[i].find_elements(By.CLASS_NAME,'ni-icon-unchecked')[j].click()
                    break
    time.sleep(10) 
    time.sleep(2)
    Number_of_jobs=driver.find_element(By.CLASS_NAME, value='styles_h1-wrapper__mHVA1').find_element(By.TAG_NAME,value='span').get_attribute("innerText")
    Modified_Number_of_jobs=Number_of_jobs[9:].strip()
   

    for i in range((int(Modified_Number_of_jobs)//20)+1):
        Jobs=driver.find_elements(By.CLASS_NAME, value='srp-jobtuple-wrapper')
        for Job in Jobs:
            Job.click()
            window_handles = driver.window_handles
            driver.switch_to.window(window_handles[-1])
            Row=driver.find_elements(By.CLASS_NAME,'styles_jhc__stat__PgY67')
            temp1=0
            temp2=0
            for i in Row:
                if "Applicants" in i.get_attribute("innerText"):
                    temp1+=1
                    Applicants.append(i.get_attribute("innerText").replace("Applicants: ",'').strip())
                if "Openings" in i.get_attribute("innerText"):
                    temp2+=1
                    Openings.append(i.get_attribute("innerText").replace("Openings: ",'').strip())
                    
            if temp1==0:
                    Applicants.append("NA")
            if temp2==0:
                    Openings.append("NA")   
            Row2=driver.find_element(By.CLASS_NAME,'styles_jd-header-comp-name__MvqAI')
            company_name=Row2.find_element(By.TAG_NAME,'a').get_attribute("innerText")
            Comp_name.append(company_name)
               
            
            driver.close()
            driver.switch_to.window(original_window)
        try:
            driver.find_element(By.LINK_TEXT,"Next").click()
        except:
            print("Done")
        time.sleep(10)
    df = pd.DataFrame({'Company': Comp_name,'Applicants': Applicants,'Openings': Openings}) 
    return df
    
    
df_front_end_blr=scrape_naukri('Bangalore','3','Frontend Engineer')   
df_back_end_blr=scrape_naukri('Bangalore','3','Backend Engineer Python')   
df_eng_man_blr=scrape_naukri('Bangalore','5','Engineering Manager') 
df_front_end1_hyd=scrape_naukri('Hyderabad','3','Frontend Engineer')   
df_back_end1_hyd=scrape_naukri('Hyderabad','3','Backend Engineer Python')  
df_eng_man1_hyd=scrape_naukri('Hyderabad','5','Engineering Manager')

Done


In [86]:
def scrape_frontend_blr(writer,df):
    df.to_excel(writer,sheet_name='Frontend Bangalore')
def scrape_backend_blr(writer,df):
    df.to_excel(writer,sheet_name='Backend Bangalore')
def scrape_engineering_blr(writer,df):
    df.to_excel(writer,sheet_name='Engineering Bangalore')
def scrape_frontend_hyd(writer,df):
    df.to_excel(writer,sheet_name='Frontend Hyderabad')
def scrape_backend_hyd(writer,df):
    df.to_excel(writer,sheet_name='Backend Hyderabad')
def scrape_engineering_hyd(writer,df):
    df.to_excel(writer,sheet_name='Engineering Hyderabad')


with pd.ExcelWriter('Job_Applicants.xlsx') as writer:
    scrape_frontend_blr(writer,df_front_end_blr)
    scrape_backend_blr(writer,df_back_end_blr)
    scrape_engineering_blr(writer,df_eng_man_blr)
    scrape_frontend_hyd(writer,df_front_end1_hyd)
    scrape_backend_hyd(writer,df_back_end1_hyd)
    scrape_engineering_hyd(writer,df_eng_man1_hyd)